In [ ]:
# For searching and accessing NASA data
import earthaccess

# For reading data, analysis and plotting
import xarray as xr
import hvplot.xarray
import geopandas as gpd

import pprint  # For nice printing of python objects

In [ ]:
earthaccess.login(strategy='interactive', persist=True)

In [ ]:
auth = earthaccess.login()

In [ ]:
query = earthaccess.search_datasets(
            keyword="GEDI",
            cloud_hosted=True
)

In [ ]:
for collection in query[:10]:
    pprint.pprint(collection.summary(), sort_dicts=True, indent=4)
    print('')  # Add a space between collections for readability

In [ ]:
gj = gpd.read_file('/home/jovyan/icepyx4gedi/files/gedi_wildfire.geojson')
bbox = tuple(list(gj.total_bounds))

In [ ]:
results = earthaccess.search_data(
    short_name = 'GEDI02_B',
    cloud_hosted = True,
    bounding_box = bbox,
    temporal = ('2019-04-04','2020-12-31')
)

In [ ]:
[display(r) for r in results[:10]]

In [ ]:
len(results)

In [ ]:
%time
files = earthaccess.open(results)
ds = xr.open_dataset(files[1], group='/BEAM0001/', phony_dims='sort')

In [ ]:
ds['pai'].hvplot(kind='scatter', s=2)

# Direct Download Approach

In [ ]:
downloaded_files = earthaccess.download(
    results[:5],
    local_path='/home/jovyan/icepyx4gedi/files/gedi/',
)

In [ ]:
from gedixr.gedi import extract_data

gedi_dir = "/home/jovyan/icepyx4gedi/files/gedi/"
gdf_l2b = extract_data(directory=gedi_dir, gedi_product='L2B',
                       subset_vector='/home/jovyan/icepyx4gedi/files/gedi_wildfire.geojson')

In [ ]:
gdf_l2b['gedi_wildfire']['gdf']

In [ ]:
from gedixr.xr import gdf_to_xr

ds = gdf_to_xr(gdf=gdf_l2b['gedi_wildfire']['gdf'])

In [ ]:
ds['pai'].plot()